Input Folder: 'final_cells'
Output Folder: 'filter_peaks'

Filters detected peaks/blobs by thresholding on Zernike moments

In [ ]:
import mahotas as mh
import numpy as np
import seaborn as sns
from colicoords import save, load
from dask.distributed import Client
from tqdm.auto import tqdm
from pathlib import Path
sns.set_style('ticks')


In [ ]:
client = Client('127.0.0.1:8786')
client

In [ ]:
d = Path('../data/final_cells').resolve()
files =[f for f in d.iterdir()]

In [ ]:
def worker(fpath):
    cells = load(fpath)
    return {fpath.stem: cells}

In [ ]:
dict_list = client.map(worker, files)

In [ ]:
out = client.gather(dict_list)
out

In [ ]:
peak_cells = {k: v for d in out for k, v in d.items()}

In [ ]:
def yield_peaks(cell, *args):
    # yields a 2k x 2x img slice around each detected peak
    k = args[0]
    xarr, yarr = cell.data.data_dict['new_thd_7500']['x'], cell.data.data_dict['new_thd_7500']['y']
    im = cell.data.data_dict['flu_mf']

    for x, y in zip(xarr, yarr):
        x = int(x)
        y = int(y)
        sel = im[y-k+1:y+k, x-k+1:x+k]
        yield sel

def yield_pole_pos(cell, *args):
    xarr, yarr = cell.data.data_dict['new_thd_7500']['x'], cell.data.data_dict['new_thd_7500']['y']
    for x, y in zip(xarr, yarr):
        lc, rc, phi = cell.coords.transform(x, y)
        if lc == 0 or lc == cell.length:
            yield True
        else:
            yield False

def yield_from_cells(cells, func, *args):
    for c in cells:
        yield from func(c, *args)

def yield_from_dict(dic, func, *args):
    for key, v in dic.items():
        yield from yield_from_cells(v, func, *args)

In [ ]:
list(peak_cells.keys())

In [ ]:
#filter peaks by thresholding their zernike moments of the 4th degree zernike polynomials
new_dict = {}
for k, v in peak_cells.items():
    c_copy = v.copy()

    for cell in tqdm(c_copy, desc=k):
        storm = cell.data.data_dict['new_thd_7500']
        imgs = np.array([im for im in yield_peaks(cell, 4)])
        if len(imgs) == 0:
            storm_yes = storm
            storm_no = storm
        else:
            zs = np.array([mh.features.zernike_moments(im, 4, degree=4, cm=(3,3)) for im in imgs])
            b = zs.T[2] > 0.4
            storm_yes = storm[b]
            storm_no = storm[~b]

        cell.data.add_data(storm_yes, 'storm', 'st_yes')
        cell.data.add_data(storm_no, 'storm', 'st_no')

    new_dict[k] = c_copy

In [ ]:
for k, v in new_dict.items():
    save('filter_peaks/' + k + '.hdf5', v)